## 环境配置

> 此为在线运行平台配置python3.9的指南，如在其他环境平台运行案例，请根据实际情况修改如下代码

第一步：设置python版本为3.9.0

In [ ]:
%%capture captured_output
!/home/ma-user/anaconda3/bin/conda create -n python-3.9.0 python=3.9.0 -y --override-channels --channel https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
!/home/ma-user/anaconda3/envs/python-3.9.0/bin/pip install ipykernel

In [ ]:
import json
import os

data = {
   "display_name": "python-3.9.0",
   "env": {
      "PATH": "/home/ma-user/anaconda3/envs/python-3.9.0/bin:/home/ma-user/anaconda3/envs/python-3.7.10/bin:/modelarts/authoring/notebook-conda/bin:/opt/conda/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/home/ma-user/modelarts/ma-cli/bin:/home/ma-user/modelarts/ma-cli/bin"
   },
   "language": "python",
   "argv": [
      "/home/ma-user/anaconda3/envs/python-3.9.0/bin/python",
      "-m",
      "ipykernel",
      "-f",
      "{connection_file}"
   ]
}

if not os.path.exists("/home/ma-user/anaconda3/share/jupyter/kernels/python-3.9.0/"):
    os.mkdir("/home/ma-user/anaconda3/share/jupyter/kernels/python-3.9.0/")

with open('/home/ma-user/anaconda3/share/jupyter/kernels/python-3.9.0/kernel.json', 'w') as f:
    json.dump(data, f, indent=4)

#### 注：以上代码运行完成后，需要重新设置kernel为python-3.9.0

<div align=center><img src="https://mindspore-demo.obs.cn-north-4.myhuaweicloud.com/imgs/ai-gallery/change-kernel.PNG"></div>

第二步：安装MindSpore框架和MindNLP套件

mindspore官网提供了不同的mindspore版本，可以根据自己的操作系统和Python版本，安装不同版本的mindspore


https://www.mindspore.cn/install

In [ ]:
%pip install https://ms-release.obs.cn-north-4.myhuaweicloud.com/2.5.0/MindSpore/unified/x86_64/mindspore-2.5.0-cp39-cp39-linux_x86_64.whl --trusted-host ms-release.obs.cn-north-4.myhuaweicloud.com -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.4/958.4 MB 9.1 MB/s eta 0:00:0000:0100:03
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 16.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 14.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 16.6 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
auto-tune 0.1.0 requires te, which is not installed.
schedule-search 0.0.1 requires absl-py, which is not installed.
Note: you may need to restart the kernel to use updated packages.


注意，这里如果通过pip安装mindnlp的话，需要参考链接对mindnlp/core/nn/modules/module.py进行Files changed所示的修改，确保loss正确下降，链接教程：https://github.com/mindspore-lab/mindnlp/pull/2007/files
可以通过git clone下载最新的mindnlp，然后将下载的最新的mindnlp版本替换到你环境中安装mindnlp的位置，一般是/home/user/miniconda3/envs/yourenv/lib/python3.9/site-packages

In [12]:
%pip install mindnlp==0.4.0 -i https://mirrors.aliyun.com/pypi/simple


Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 4.0 MB/s eta 0:00:00a 0:00:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 15.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for jieba: filename=jieba-0.42.1-py3-none-any.whl size=19314508 sha256=30064bba508d12a9c2c545bdec7e271f61d5a83e9fdd53298a82e74659e1fd26
  Stored in directory: /home/jiangna1/.cache/pip/wheels/95/ef/7c/d8b3108835edfa15487417c5bddff166482b195d8090117ac5
Successfully built jieba
Note: you may need to restart the kernel to use updated packages.


## LLaMA微调推理全流程

# LLaMA介绍

LLaMA（Large Language Model Meta AI）是由Meta（前Facebook）推出的一系列开源大规模语言模型，意味着大模型应用进入了“免费时代”，初创公司也能够以低廉的价格来创建类似ChatGPT这样的聊天机器人。 LLaMA 专注于自然语言处理任务，其结构基于Transformer架构，但是进行了改进，比如引入更高效的注意力机制和更紧凑的模型设计，例如使用SwiGLU激活函数 使用RoPE位置编码等。

LLaMA的模型参数规模设计更为精细，以更少的参数实现了与更大模型相当的性能。例如，LLaMA-7B模型的性能可以与175B参数规模的GPT-3媲美。模型在推理过程中采用了多查询注意力（Multi-Query Attention, MQA）机制，改进了传统多头注意力的查询方式，将多个注意力头的查询统一为单个查询头，从而显著减少了推理时间和显存需求，提升了效率。适用于文本生成、问答、翻译等任务。例如，在文本生成任务中，LLaMA可以生成高质量的文章段落，其轻量化设计降低了硬件需求，使研究者和开发者更容易使用高性能的语言模型。

导入必要的包

In [3]:
import json
import numpy as np
import mindspore as ms
import mindspore.dataset as ds

In [4]:
#将模式设置为动态图模式（PYNATIVE_MODE），并指定设备目标为Ascend芯片
ms.set_context(mode=ms.PYNATIVE_MODE, device_target="Ascend")


[WARNING] ME(52617:140460371482432,MainProcess):2025-04-11-09:21:14.572.566 [mindspore/context.py:1335] For 'context.set_context', the parameter 'device_target' will be deprecated and removed in a future version. Please use the api mindspore.set_device() instead.


In [5]:
#指定模型路径
base_model_path = "/home/jiangna1/.mindnlp/model/hfl/chinese-llama-2-1.3b" #中文模型，这里我提前下载到了本地减少下载时间
# base_model_path = "NousResearch/Hermes-3-Llama-3.2-3B" #英文模型



## 数据集

这里提供两份用于微调的数据集，分别用于中文模型和英文模型，其中中文模型为hfl/chinese-llama-2-1.3b，参数量为1.3B，英文模型为NousResearch/Hermes-3-Llama-3.2-3B，参数量为3.2B，用户可以根据自己的配置或期望训练时长自行选择。

数据来源皆为hugging face公开用于微调的数据集，其中中文数据集来源为弱智吧，数据格式为：

  {"instruction": "只剩一个心脏了还能活吗？",

    "output": "能，人本来就只有一个心脏。"},

  {"instruction": "爸爸再婚，我是不是就有了个新娘？",

    "output": "不是的，你有了一个继母。\"新娘\"是指新婚的女方，而你爸爸再婚，他的新婚妻子对你来说是继母。"}


英文数据来源为Alpaca,数据格式为：

  {"instruction": "Give three tips for staying healthy.",

    "input": "",

    "output": "1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule."},

  {"instruction": "What are the three primary colors?",

    "input": "",

    "output": "The three primary colors are red, blue, and yellow."}


以下教程同时包括包括中文和英文模型的微调教程为例，其中英文模型微调效果更好，但因为时间关系，本模型展示主要以小规模的中文为例，可以自行根据自己的需求修改数据来源和模型。
 

### 数据加载和数据预处理

新建 tokenize_function 函数用于数据预处理，具体内容可见下面代码注释。

In [6]:
def tokenize_function(example, tokenizer):
    instruction = example.get("instruction", "")
    input_text = example.get("input", "")
    output = example.get("output", "")
    # prompt
    if input_text:
        prompt = f"User: {instruction} {input_text}\nAssistant: {output}"
    else:
        prompt = f"User: {instruction}\nAssistant: {output}"
    
    # Tokenize
    tokenized = tokenizer(prompt, padding="max_length", truncation=True, max_length=512)
    input_ids = np.array(tokenized["input_ids"], dtype=np.int32)

    # Handle label
    pad_token_id = tokenizer.pad_token_id
    labels = np.array(
        [-100 if token_id == pad_token_id else token_id for token_id in input_ids], dtype=np.int32
    )
    return input_ids, labels



数据来源如下，为了避免网络问题，建议先下载到本地

https://huggingface.co/datasets/LooksJuicy/ruozhiba

In [7]:
# data_path = "/home/jiangna1/mindnlp_llama_all/alpaca_data.json" #英文数据集
data_path = "/home/jiangna1/mindnlp_llama_all/chinese_data.json"

查看数据具体内容，该数据只包括instruction和output两列

In [8]:
with open(data_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
print(data[:3])

[{'instruction': '只剩一个心脏了还能活吗？', 'output': '能，人本来就只有一个心脏。'}, {'instruction': '爸爸再婚，我是不是就有了个新娘？', 'output': '不是的，你有了一个继母。"新娘"是指新婚的女方，而你爸爸再婚，他的新婚妻子对你来说是继母。'}, {'instruction': '樟脑丸是我吃过最难吃的硬糖有奇怪的味道怎么还有人买', 'output': '樟脑丸并不是硬糖，而是一种常见的驱虫药，不能食用。虽然它的味道可能不太好，但是由于其有效的驱虫效果，所以仍然有很多人会购买。'}]


从指定路径加载预训练的分词器，该分词器能将输入文本分割成模型可处理的词元。接着，将填充标记设置为结束标记，这样在处理不同长度的文本序列时，用结束标记来填充额外位置，避免引入额外特殊标记，减少模型学习负担。最后，设置填充方向为右侧，使文本在右侧添加填充标记达到统一长度，维持文本原始顺序。

In [9]:
from mindnlp.transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(base_model_path)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"
tokenizer.encode(' ;')


[1, 29871, 2056]

将数据分为训练集和验证集

In [10]:

def data_generator(dataset, tokenizer):
    for item in dataset:
        yield tokenize_function(item, tokenizer)
        
split_ratio = 0.9
split_index = int(len(data) * split_ratio)
train_data, val_data = data[:split_index], data[split_index:]

train_dataset = ds.GeneratorDataset(
    source=lambda: data_generator(train_data, tokenizer), 
    column_names=["input_ids", "labels"]
)

eval_dataset = ds.GeneratorDataset(
    source=lambda: data_generator(val_data, tokenizer), 
    column_names=["input_ids", "labels"]
)

查看处理后的数据，tokenizer 将输入的文本（prompt）拆分为词片段（tokens），然后将每个词片段映射为对应的 token ID。

In [11]:
for i, sample in enumerate(train_dataset.create_dict_iterator()):
    if i >= 5:
        break
    print(f"Sample {i}: Input IDs: {sample['input_ids'][:10]}") 
    print(f"Sample {i}: Labels: {sample['labels'][:10]}\n")   

[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB
Sample 0: Input IDs: [    1  4911 29901 29871 47133 32002 37755 30743 33302 31704]
Sample 0: Labels: [    1  4911 29901 29871 47133 32002 37755 30743 33302 31704]

Sample 1: Input IDs: [    1  4911 29901 29871 33594 31733 33364 30214 30672 32308]
Sample 1: Labels: [    1  4911 29901 29871 33594 31733 33364 30214 30672 32308]

Sample 2: Input IDs: [    1  4911 29901 29871 47019 33027 31818 34030 39950 44345]
Sample 2: Labels: [    1  4911 29901 29871 47019 33027 31818 34030 39950 44345]

Sample 3: Input IDs: [    1  4911 29901 29871 34214 30698 30429 36310 32658 30743]
Sample 3: Labels: [    1  4911 29901 29871 34214 30698 30429 36310 32658 30743]

Sample 4: Input IDs: [    1  4911 29901 32581 34822 31639  2882  6530 30883 30210]
Sample 4: Labels: [    1  4911 29901 32581 34822 31639  2882  6530 30883 30210]



## lora指令微调

指定微调结果输出路径

In [12]:
# 指定输出路径
peft_output_dir = "/home/jiangna1/mindnlp_llama_all/pert_model_Chinese"

加载基座模型


In [13]:
from mindnlp.transformers import AutoModelForCausalLM,  GenerationConfig

ms_base_model = AutoModelForCausalLM.from_pretrained(base_model_path, ms_dtype=ms.float16)
ms_base_model.generation_config = GenerationConfig.from_pretrained(base_model_path)
ms_base_model.generation_config.pad_token_id = ms_base_model.generation_config.eos_token_id


LlamaForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`.`PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


/home/jiangna1/miniconda3/envs/ms39/lib/python3.9/site-packages/mindnlp/transformers/generation/configuration_utils.py:557: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/jiangna1/miniconda3/envs/ms39/lib/python3.9/site-packages/mindnlp/transformers/generation/configuration_utils.py:562: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/jiangn

In [14]:
#修改精度，会使训练变慢，但是训练loss下降效果会变好
for name, param in ms_base_model.parameters_and_names():
    param.set_dtype(ms.float32) 

这部分代码的主要作用是创建一个 LoRA的配置对象 ms_config，大语言模型进行微调时，LoRA 是一种高效的参数微调方法，通过在预训练模型的基础上添加低秩矩阵来减少需要训练的参数数量，从而提高微调效率。

In [15]:
from mindnlp.peft import LoraConfig, TaskType, get_peft_model, PeftModel

ms_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,#微调任务的类型
    #指定需要应用 LoRA 调整的目标模块
    # target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    inference_mode=False,  
    r=8,                   
    lora_alpha=32,         
    lora_dropout=0.1       
)


基于给定的基础模型和 LoRA 配置创建一个可进行参数高效微调的模型。

In [16]:
ms_model = get_peft_model(ms_base_model, ms_config)

训练参数的设置

In [17]:
from mindnlp.engine import TrainingArguments, Trainer

num_train_epochs = 70
fp16 = True
overwrite_output_dir = True
per_device_train_batch_size = 16
per_device_eval_batch_size = 32
gradient_accumulation_steps = 16
gradient_checkpointing = True
evaluation_strategy = "steps"
learning_rate = 1e-5
lr_scheduler_type = "cosine" 
weight_decay = 0.01
warmup_ratio = 0.1
max_grad_norm = 0.3
group_by_length = False 
auto_find_batch_size = False
save_steps = 50 
logging_strategy = "steps"
logging_steps = 150                
load_best_model_at_end = True 
packing = False
save_total_limit = 3
neftune_noise_alpha = 5 
eval_steps = 10

training_arguments = TrainingArguments(
    output_dir=peft_output_dir,
    overwrite_output_dir=overwrite_output_dir,
    num_train_epochs=num_train_epochs,
    load_best_model_at_end=load_best_model_at_end,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    evaluation_strategy=evaluation_strategy,
    eval_steps=eval_steps,
    max_grad_norm=max_grad_norm,
    auto_find_batch_size=auto_find_batch_size,
    save_total_limit=save_total_limit,
    gradient_accumulation_steps=gradient_accumulation_steps,
    save_steps=save_steps,
    logging_strategy=logging_strategy,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type
)

初始化训练器

In [16]:
trainer = Trainer(
    model=ms_model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    args=training_arguments
)

开始训练

In [ ]:
trainer.train()




  0%|          | 1/350 [00:19<1:51:45, 19.21s/it]

.

  3%|▎         | 10/350 [02:02<1:04:58, 11.47s/it]We detected that you are passing `past_key_values` as a tuple and this is deprecated. Please use an appropriate `Cache` class
                                                  
  3%|▎         | 10/350 [02:05<1:04:58, 11.47s/it]

{'eval_loss': 3.853940486907959, 'eval_runtime': 2.8532, 'eval_samples_per_second': 1.752, 'eval_steps_per_second': 0.35, 'epoch': 1.88}


                                                  
  6%|▌         | 20/350 [04:00<1:02:29, 11.36s/it]

...

{'eval_loss': 3.0396060943603516, 'eval_runtime': 2.3904, 'eval_samples_per_second': 2.092, 'eval_steps_per_second': 0.418, 'epoch': 64.0}


                                                   
100%|██████████| 350/350 [1:08:03<00:00, 11.35s/it]

{'eval_loss': 3.039623260498047, 'eval_runtime': 2.3877, 'eval_samples_per_second': 2.094, 'eval_steps_per_second': 0.419, 'epoch': 65.88}


The intermediate checkpoints of PEFT may not be saved correctly, consider using a custom callback to save adapter_model.bin in corresponding saving folders. Check some examples here: https://github.com/huggingface/peft/issues/96
100%|██████████| 350/350 [1:08:12<00:00, 11.69s/it]


{'train_runtime': 4092.112, 'train_samples_per_second': 23.264, 'train_steps_per_second': 0.086, 'train_loss': 3.2515819876534597, 'epoch': 65.88}


In [ ]:
for name, param in trainer.model.parameters_and_names():
    param.set_dtype(ms.float16)
trainer.model.save_pretrained(peft_output_dir)

最后存储还是存储为 16 位浮点数，保存训练后的 LoRA 模型保存到指定的输出目录

## 推理

注意，lora微调后保存的并不是完整的参数，在推理时，需要将保存的 LoRA 参数加载到原预训练模型中，合并后得到完整的模型，然后进行推理。

使用PeftModel进行配置和参数合并，最后将模型设置为评估模式，进行推理任务。

In [18]:
#将 LoRA微调后的参数加载到预训练模型中
from mindnlp.peft import PeftModel
model = PeftModel.from_pretrained(ms_base_model, peft_output_dir)
model = model.merge_and_unload()
print('model merge succeeded')
model.eval()


model merge succeeded


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(55296, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-3): 4 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear (4096 -> 4096)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear (4096 -> 8)
            )
            (lora_B): ModuleDict(
              (default): Linear (8 -> 4096)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear(
            (base_layer): Linear (4096 -> 4096)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear (4096 -> 8)
  

定义一个函数，用于根据用户输入的问题生成相应的回答

In [21]:
import mindspore as ms

def generate_response(question, model, tokenizer, max_length=256):
    prompt = f"以下是用户和助手之间的问答。\n问：{question}\n答："
    inputs = tokenizer(prompt, return_tensors="ms", padding=True, truncation=True, max_length=512)
    output_ids = model.generate(
        **inputs,
        do_sample=False,
        # temperature=0.7,
        # top_p=0.9,
        repetition_penalty=1.2,
        no_repeat_ngram_size=3,
        max_length=max_length,
        eos_token_id=tokenizer.eos_token_id
    )

    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    response = response.split("Answer：")[-1].strip()
    return response


一个实例

In [22]:
question = "如何保持清醒?"
response = generate_response(question, model, tokenizer)

print(f"User: {question}")
print(f"LLAMA: {response}")

/home/jiangna1/miniconda3/envs/ms39/lib/python3.9/site-packages/mindnlp/transformers/generation/configuration_utils.py:557: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/jiangna1/miniconda3/envs/ms39/lib/python3.9/site-packages/mindnlp/transformers/generation/configuration_utils.py:562: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


User: 如何保持清醒?
LLAMA: 以下是用户和助手之间的问答。
问：如何保持清醒?
答：在你睡觉的时候，你的大脑会一直处于兴奋状态中;当你醒来时，它就会继续工作了。所以如果你的睡眠时间很短的话，你就不会感到太疲劳或昏沉。你可以通过使用一些药物来帮助恢复精力、提高警觉度以及降低血压等方法使自己进入深度睡眠的状态。此外，你还可以通过服用维生素B6片剂或者吃富含蛋白质的食物等方式让自己重新振作起来。
